## Import libraries

In [ ]:
# generic tools
import numpy as np
import datetime 

# tools from sklearn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# tools from tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model

# matplotlib
import matplotlib.pyplot as plt

# Load the TensorBoard notebook extension
%load_ext tensorboard
# delete logs from previous runs - not always safe!
!rm -rf ./logs/

## Download data, train-test split, binarize labels

## Define neural network architecture using ```tf.keras```

## Show summary of model architecture

## Visualise model layers

## Compile model loss function, optimizer, and preferred metrics

## Train model and save history

## Visualise using ```matplotlib```

## Inspect using ```tensorboard```

We can inspect the results in lots of detail by using ```Tensorboard```. 

If you're working on your own machine, you can do this directly in a notebook. On UCloud, though, we need to follow the instructions set out [here](https://tensorboard.dev/)

## Classifier metrics